In [21]:
from bs4 import BeautifulSoup
import os
import requests
import re
from lyricsgenius import Genius

In [22]:
GENIUS_API_TOKEN='bVEbboB9VeToZE48RaiJwrnAGLz8VbrIdlqnVU70pzJXs_T4Yg6pdPpJrTQDK46p'
token = GENIUS_API_TOKEN

In [23]:
# Approach 1, use abstraction library lyricsgenius https://github.com/johnwmillr/LyricsGenius
genius = Genius(token)
artist = genius.search_artist("Andy Shauf", max_songs=3, sort="title")
song = artist.song("To You")
song.lyrics

Searching for songs by Andy Shauf...

Song 1: "Alexander All Alone"
Song 2: "All the Same"
Song 3: "Angela"

Reached user-specified song limit (3).
Done. Found 3 songs.
Searching for "To You" by Andy Shauf...
Done.


"Jeremy can we talk a minute?\nI've got some things that I need to\nGet off of my chest\n\nI know that we have had a few\nAnd it's far too late\nBut if I wait I might never tell you\n\nCan we find somewhere quiet?\nLet's go outside\nI've got some smokes if you've got a light\n\nIt's just that sometimes when I'm by your side\nIt feels so right\nIt feels like nothing could go wrong\n\nDoes it ever\nFeel like that to you?\nDoes it ever\nFeel like that to you?\n\nOh I don't know what I mean\nIt sounded wrong and I'm just tired\nNot being weird\n\nI just mean that you're a good friend\nIt's hard to explain\nJust forget I said anything\n\nGet over yourself\nI'm not in love with you\nIt just came out all wrong\n\nYeah\nTell the guys and laugh it up\nWhy am I even surprised?\n\nThat it never\nFeels like that to you\nThat it never\nFeels like that to you\nI guess it never\nFeels like that to you\nI guess it never\nFeels like that to you"

In [24]:
# Approach 2, use Genius API directly https://medium.com/analytics-vidhya/how-to-scrape-song-lyrics-a-gentle-python-tutorial-5b1d4ab351d2
# Get artist object from Genius API
def request_artist_info(artist_name, page):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    return response
# Get Genius.com song url's from artist object
def request_song_url(artist_name, song_cap):
    page = 1
    songs = []
    
    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()
        # Collect up to song_cap song objects from artist
        song_info = []
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
    
        # Collect song URL's from song objects
        for song in song_info:
            if (len(songs) < song_cap):
                url = song['result']['url']
                songs.append(url)
            
        if (len(songs) == song_cap):
            break
        else:
            page += 1
        
    print('Found {} songs by {}'.format(len(songs), artist_name))
    return songs
    
# DEMO
request_song_url('Andy Shauf', 3)


Found 3 songs by Andy Shauf


['https://genius.com/Andy-shauf-the-magician-lyrics',
 'https://genius.com/Andy-shauf-quite-like-you-lyrics',
 'https://genius.com/Andy-shauf-try-again-lyrics']

In [19]:
# Scrape lyrics from a Genius.com song URL
def scrape_song_lyrics(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics = html.find('div', class_='lyrics').get_text()
    #remove identifiers like chorus, verse, etc
    lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
    #remove empty lines
    lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])         
    return lyrics
# DEMO
print(scrape_song_lyrics('https://genius.com/Andy-shauf-to-you-lyrics'))

Jeremy can we talk a minute?
I've got some things that I need to
Get off of my chest
I know that we have had a few
And it's far too late
But if I wait I might never tell you
Can we find somewhere quiet?
Let's go outside
I've got some smokes if you've got a light
It's just that sometimes when I'm by your side
It feels so right
It feels like nothing could go wrong
Does it ever
Feel like that to you?
Does it ever
Feel like that to you?
Oh I don't know what I mean
It sounded wrong and I'm just tired
Not being weird
I just mean that you're a good friend
It's hard to explain
Just forget I said anything
Get over yourself
I'm not in love with you
It just came out all wrong
Yeah
Tell the guys and laugh it up
Why am I even surprised?
That it never
Feels like that to you
That it never
Feels like that to you
I guess it never
Feels like that to you
I guess it never
Feels like that to you
